In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import integrate
from matplotlib import pyplot as plt

In [ ]:
sns.set_context('poster')

Each time a CR proton ionises an H atom, an electron with average energy ⟨E⟩=35eV is produced (Spitzer et al., 1968). Including the ionization energy of 13.6eV, the CR proton loses approximately 50eV per scattering. This necessarily places a limit on how many scatterings a CR proton can undergo before losing all its energy to ionisation, as well as limiting the distance it may travel. This distance may be described by a penetration depth
$$ D_p(n, \epsilon) \approx \frac{\beta c \epsilon} {-({\rm d}\epsilon / {\rm d}t)_{\rm ion}}$$

In [ ]:
def Dp(n,epsilon):
    return beta(epsilon) * 2.99e10 * epsilon / dedt(n,epsilon)

where, according to Schlickeiser (2002):
$$\beta =  \sqrt{1 - \left( \frac{\epsilon}{m_{\rm \tiny H}c^2}+1 \right)^{-2}},$$

In [ ]:
def beta(epsilon):
    mHc2 = 938272046 #rest energy of a proton in eV
    return np.sqrt(1 - (epsilon/mHc2 + 1)**(-2))

$$f(\epsilon) = (1 + 0.0185 \,{\rm ln}\beta )\, \frac{2 \beta^2}{\beta_0^3 + 2 \beta^3},$$

In [ ]:
def f(epsilon):
    B0 = 0.01
    B = beta(epsilon)
    return (1 + 0.0185 * np.log(B)) * 2*B**2 / (B0**3 + 2*B**3)

and $$- \left( \frac{{\rm d}\epsilon} {{\rm d}t} \right)_{\rm ion}(n, \epsilon) = 1.82\times10^{-7}\,{\rm \small eV\,s}^{-1} n_{\rm \tiny H} f(\epsilon).$$

In [ ]:
def dedt(n, epsilon):
    return 1.82e-7 * n * f(epsilon)

Here, $m_{\rm \small H}$ is the mass of hydrogen, $c$ is the speed of light, $\beta = v/c$ and $({\rm d}\epsilon / {\rm d}t)_{\rm ion}$ is the rate at which a CR proton loses energy to ionisation. $\beta_0$ is the cutoff below which the interaction between CRs and the gas decreases sharply; we use $\beta_0=0.01$, appropriate for CRs traveling through a neutral IGM (Stacy et al., 2007). As $D_p(n, \epsilon)$ is the the mean free path of CRs of energy $\epsilon$ traveling through a gas with number density $n$, we may define an effective cross-section $\sigma_{CR}(n,\epsilon)$ for the interaction: $$\sigma_{CR}(n,\epsilon) = \frac{1}{n D_p(n, \epsilon)}.$$

In [ ]:
def sigma(n, epsilon):
    return 1 / n / Dp(n,epsilon)

This in turn allows us to define an optical depth
$$\tau_{CR}(n,\epsilon) = N \sigma_{CR}(n,\epsilon)$$
where $N$ is the gas column density.

In [ ]:
def old_shielding(n, epsilon):
    Omega_pole = 1.840282
    fourpi = 4 * np.pi
    a = 2 * Omega_pole / fourpi
    b = (fourpi - 2*Omega_pole)/fourpi
    def N_eff(n):
        x = N_p(n)
        y = N_eq(n)
        return a*x + b*y
    return np.exp(-sigma(n, epsilon) * N_eff(n))

In [ ]:
def pole_shielding(n, epsilon):
    return np.exp(-sigma(n, epsilon) * N_p(n))

In [ ]:
def eq_shielding(n, epsilon):
    return np.exp(-sigma(n, epsilon) * N_eq(n))

In [ ]:
def shielding(n, epsilon):
    Omega_pole = 1.840282
    fourpi = 4 * np.pi
    a = 2 * Omega_pole / fourpi
    b = (fourpi - 2*Omega_pole)/fourpi
    return a * np.exp(-sigma(n, epsilon) * N_p(n)) + b * np.exp(-sigma(n, epsilon) * N_eq(n))

Following Stacy et al. (2007), we assume cosmic rays in the early universe are produced in supernova shock waves, resulting in a differential CR energy spectrum of the form 
$$ \frac{{\rm d}n_{\rm \small CR}}{{\rm d}\epsilon} = \frac{n_{\rm norm}}{\epsilon_{\rm min}}\left( \frac{\epsilon}{\epsilon_{\rm min}} \right)^{-2},$$

where $n_{\rm \small CR}$ is the CR number density, $\epsilon$ is the kinetic energy of the CR, $\epsilon_{\rm min}$ is the low-energy cutoff of the CR spectrum, and $n_{\rm norm}$ is a normalizing density factor. Integrating this over all energies yields the total energy density $u_{\rm \small CR}$ in cosmic rays, which we estimate as follows:

$$u_{\rm \small CR}(z) = f_{\rm \small CR} E_{\rm \small SN}\, f_{\rm \small SN} \Psi_{*}(z)\, t_{\rm \small H}(z) (1+z)^3.$$

As describe [here](https://grad09jh.as.utexas.edu:7777/notebooks/cosmic_rays/The%20High-z%20Cosmic%20Ray%20Background.ipynb), $f_{\rm \small CR}$ is the fraction of the SN explosion energy $E_{\rm \small SN}$ going into CR production, $f_{\rm \small SN}$ is the mass fraction of stars formed which die as SNe, and $\Psi_{*}(z)$ is the comoving star formation rate density (SFRD) as a function of redshift.  The Hubble time $t_{\rm \small H}$ accounts for the time CRs have had to propagate through the universe since their creation, and the factor of $(1+z)^3$ accounts for the conversion from a comoving SFRD to a physical energy density.   This results in a CR energy spectrum increasing over cosmic time in the following fashion:
$$ \frac{{\rm d}n_{\rm \small CR}}{{\rm d}\epsilon}(z) = \frac{u_{\rm \small CR}(z)}{{\rm ln}\,\epsilon_{\rm max}{\large /}\epsilon_{\rm min}}  \epsilon^{-2}.$$

In [ ]:
def dnCR(epsilon, ucr, emin, emax):
    return ucr / epsilon / epsilon / np.log(emax/emin)

#Calculate $N_{\rm \small effective}$ from $N_{\rm \small pole}$ and $N_{\rm \small equator}$.

###$N_{\rm \small effective} = N_{\rm \small pole} \frac{2 \Omega_{\rm \small pole}}{4\pi} + N_{\rm \small equator} \frac{4\pi - 2 \Omega_{\rm \small pole}}{4\pi}$

###$\Omega_{\rm \small pole} = \int_0^{2\pi}{\rm d}\phi \int_0^{\pi/4}{\rm sin}\theta \,{\rm d}\theta  = 2\pi \left( 1 - \frac{1}{\sqrt{2}} \right) \approx 2\pi \times 0.29289 = 1.840282$

#From our [fit](Local Optical Depth Prescription.ipynb),
##$N_{\rm \small pole} = 10^{0.532301 {\rm log_{10}}(n) + 19.636552}$ and $N_{\rm \small equator} = 10^{0.626204 {\rm log_{10}}(n) + 19.573490}$

In [ ]:
def N_p(n):
    exponent = 0.532301 * np.log10(n) + 19.636552
    return 10**exponent

def N_eq(n):
    exponent = 0.626204 * np.log10(n) + 19.573490
    return 10**exponent

In [ ]:
def integrand(epsilon, n, ucr, emin, emax, attenuation=False):
    if attenuation:
        if attenuation == 'corrected':
            return .02 * dedt(n,epsilon) * dnCR(epsilon, ucr, emin, emax) * shielding(n, epsilon)
        elif attenuation == 'old':
            return .02 * dedt(n,epsilon) * dnCR(epsilon, ucr, emin, emax) * old_shielding(n, epsilon)
        elif attenuation == 'pole':
            return .02 * dedt(n,epsilon) * dnCR(epsilon, ucr, emin, emax) * pole_shielding(n, epsilon)
        elif attenuation == 'eq':
            return .02 * dedt(n,epsilon) * dnCR(epsilon, ucr, emin, emax) * eq_shielding(n, epsilon)
    else:
        return .02 * dedt(n,epsilon) * dnCR(epsilon, ucr, emin, emax)

In [ ]:
def logintegrate(func, n, ucr, emin, emax, attenuation=True, n_steps=1000):
    #print 'emin: 1e'+str(np.log10(emin)), 'emax: 1e'+str(np.log10(emax)),
    #print 'n:',n, 'ucr:',ucr, 'tau:',attenuation
    integral = 0.0
    logmin = np.log10(emin)
    logmax = np.log10(emax)
    for i in xrange(n_steps):
        epsilon = (logmax - logmin) / n_steps * (i + 0.5) + logmin
        e_start = (logmax - logmin) / n_steps * (i) + logmin
        e_stop = (logmax - logmin) / n_steps * (i + 1.0) + logmin
        epsilon = 10**epsilon
        e_start = 10**e_start
        e_stop = 10**e_stop
        De = e_stop - e_start
        integral += func(epsilon, n, ucr, emin, emax, attenuation) * De
    return integral

In [ ]:
def oldintegrand(epsilon, n, ucr, emin, emax, attenuation):
    return f(epsilon) / epsilon / epsilon / np.log(emax/emin)
logintegrate(oldintegrand, 1, 1, 1e6, 1e15, False)  * 1.82e-7 / 50

In [ ]:
@np.vectorize
def h_rate(n, ucr, emin, emax, attenuation=True):
    E_heat = 6 / 6.24150934e11 # convert from eV to erg
    return E_heat * logintegrate(integrand, n, ucr, emin, emax, attenuation)

@np.vectorize
def k_rate(n, ucr, emin, emax, attenuation=True):
    return logintegrate(integrand, n, ucr, emin, emax, attenuation) / n

In [ ]:
k_rate(1, 1, 1e6, 1e15, 'corrected')

In [ ]:
nstart = -4
nstop = 12.2
nstep = 82
n = np.logspace(nstart,nstop,nstep)

In [ ]:
ucr = 1.
E_min = 1e6
E_max = 1e15
attenuation = 'corrected'
kh_rates = pd.DataFrame(index=np.linspace(nstart,nstop,nstep))
kh_rates['ion'] = k_rate(n, ucr, E_min, E_max, attenuation)
kh_rates['kHe'] = 0
kh_rates['kHep'] = 0
kh_rates['heat'] = h_rate(n, ucr, E_min, E_max, attenuation) / n 
kh_rates['hHe'] = 0
kh_rates['hHep'] = 0
kh_rates['nheat'] = h_rate(n, ucr, E_min, E_max, attenuation)

In [ ]:
ucr = 1.
E_min = 1e6
E_max = 1e15
attenuation = 'old'
old_kh_rates = pd.DataFrame(index=np.linspace(nstart,nstop,nstep))
old_kh_rates['ion'] = k_rate(n, ucr, E_min, E_max, attenuation)
old_kh_rates['kHe'] = 0
old_kh_rates['kHep'] = 0
old_kh_rates['heat'] = h_rate(n, ucr, E_min, E_max, attenuation) / n 
old_kh_rates['hHe'] = 0
old_kh_rates['hHep'] = 0
old_kh_rates['nheat'] = h_rate(n, ucr, E_min, E_max, attenuation)

In [ ]:
ucr = 1.
E_min = 1e6
E_max = 1e15
attenuation = 'pole'
pole_kh_rates = pd.DataFrame(index=np.linspace(nstart,nstop,nstep))
pole_kh_rates['ion'] = k_rate(n, ucr, E_min, E_max, attenuation)
pole_kh_rates['kHe'] = 0
pole_kh_rates['kHep'] = 0
pole_kh_rates['heat'] = h_rate(n, ucr, E_min, E_max, attenuation) / n 
pole_kh_rates['hHe'] = 0
pole_kh_rates['hHep'] = 0
pole_kh_rates['nheat'] = h_rate(n, ucr, E_min, E_max, attenuation)

In [ ]:
ucr = 1.
E_min = 1e6
E_max = 1e15
attenuation = 'eq'
eq_kh_rates = pd.DataFrame(index=np.linspace(nstart,nstop,nstep))
eq_kh_rates['ion'] = k_rate(n, ucr, E_min, E_max, attenuation)
eq_kh_rates['kHe'] = 0
eq_kh_rates['kHep'] = 0
eq_kh_rates['heat'] = h_rate(n, ucr, E_min, E_max, attenuation) / n 
eq_kh_rates['hHe'] = 0
eq_kh_rates['hHep'] = 0
eq_kh_rates['nheat'] = h_rate(n, ucr, E_min, E_max, attenuation)

In [ ]:
attenuation = False
khr = pd.DataFrame(index=np.linspace(nstart,nstop,nstep))
khr['ion'] = k_rate(n, ucr, E_min, E_max, attenuation)
khr['heat'] = h_rate(n, ucr, E_min, E_max, attenuation) / n
khr['nheat'] = h_rate(n, ucr, E_min, E_max, attenuation)

In [ ]:
kh_rates.head(3)

In [ ]:
kh_rates.tail(3)

In [ ]:
plt.plot(kh_rates.index, kh_rates.ion, label='corrected')
plt.plot(eq_kh_rates.index, eq_kh_rates.ion, '--', label='equatorial')
plt.plot(pole_kh_rates.index, pole_kh_rates.ion, '--', label='polar')
plt.plot(old_kh_rates.index, old_kh_rates.ion, ':', color='grey', label='column avg')
plt.plot(khr.index, khr.ion, '--', color='grey', label='no attenuation')
plt.yscale('log')
plt.ylim(1e-20)
plt.xlim(-4,12)
plt.legend(loc=0)
plt.xlabel('Log n [cm$^{-3}$]')
plt.ylabel('Ionization Rate [s$^{-1}$]')
plt.savefig('figures/khrates/ionrate.png', bbox_inches='tight')

In [ ]:
kh_rates['heat'].plot(logy=True)
plt.plot(khr.index, khr.heat, '--', color='grey')
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('Heating Rate / n [erg s$^{-1}$]')
plt.savefig('figures/khrates/hrate.png', bbox_inches='tight')

In [ ]:
kh_rates['nheat'].plot(logy=True)
plt.plot(khr.index, khr.nheat, '--', color='grey')
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('Heating Rate [erg s$^{-1}$ cm$^{-3}$]')
plt.savefig('figures/khrates/nhrate.png', bbox_inches='tight')

In [ ]:
kh_data = np.column_stack((kh_rates.index.values, kh_rates[['ion', 'kHe', 'kHep', 'heat', 'hHe', 'hHep']].values))

In [ ]:
kh_data.shape

In [ ]:
np.savetxt('kh_rates.dat', kh_data, fmt='%.8e %.8e %.8e %.8e %.8e %.8e %.8e')

In [ ]:
!head kh_rates.dat

In [ ]:
!tail kh_rates.dat